In [ ]:
# pandas 의 분석기법
- 통계처리 함수 : mean(평균), var(분산) , std(standard deviation,표준편차),cov(공분산),corr(상관관계 -1~ 1 )
- 데이터요약 : groupby, cross_table(교차분석표-카운트), pivot(값이 변하지않음) ,pivot_table(함수)
    -aggregation,filter,transform,apply 
- 시간과 시계열 분석의 전처리 도구
    - 자기상관성 (auto corelation)
    -rnn 을 이해할 때 도움이 됨 (시간 순서를 가진 : 언어,음성,)
    - shift(이동), diff(차분-미분) , rolling(이동 평균법),resampling(sample의 개수를 맞춘다) ,pct_change 
    - window

In [ ]:
# 범주형, 연속형 변수 
범주형(분류,상중하 어디에속하냐) > 남자, 여자 or 상 , 중 , 하 등 이산적 데이터
연속형 변수(예측,부동산가격,키의예측) > 키 172.5 , 180 등 
- 독립변수 연속형 , 종속변수가 범주형 : logistic regression(로지스틱회귀) > 회귀분석
            - 소득, 부채 > 대출여부(Y/N) 

- 독립변수가 연속형 , 종속변수도 연속형 : 회귀분석, 상관분석 
    - 독립 : 공단에서의 거리, 고속도로의 거리, 방의 개수 .... , 종속변수는 : 주택의 가격

In [4]:
# 범주형인 경우
import pandas as pd 
import numpy as np
# 계층적 인덱스
arrays = [['bar','bar','baz','baz','foo','foo','qux','qux'],['one','two','one','two','one','two','one','two']]
index = pd.MultiIndex.from_arrays(arrays, names=['first','second'])
s = pd.Series(np.random.randn(8), index = index)
s


first  second
bar    one      -0.994001
       two       0.550212
baz    one      -0.681331
       two      -0.439750
foo    one      -0.841534
       two       0.453015
qux    one      -0.362622
       two      -0.566235
dtype: float64

In [5]:
grouped = s.groupby(level=0)
grouped.sum()

first
bar   -0.443789
baz   -1.121081
foo   -0.388519
qux   -0.928858
dtype: float64

In [7]:
s.groupby(level='second').sum()

second
one   -2.879489
two   -0.002759
dtype: float64

In [8]:
s.groupby(level=['first','second']).sum()

first  second
bar    one      -0.994001
       two       0.550212
baz    one      -0.681331
       two      -0.439750
foo    one      -0.841534
       two       0.453015
qux    one      -0.362622
       two      -0.566235
dtype: float64

In [39]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
           ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
df = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3],
                   'B': np.arange(8)},
                  index=index)
df 

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [40]:
df.groupby([pd.Grouper(level=1),'A']).sum() # 그룹으로 묶이고 , 계산은 집계함수

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [41]:
index = pd.date_range('10/1/1999', periods=1100)# 주기로 (디폴트 주기는 일)
print(index)
ts = pd.Series(np.random.normal(0.5, 2, 1100), index) # 11100개 데이터  # 평균> 0.5 표준편차 2 인 범위에서 1100개의 데이터
ts = ts.rolling(window=100, min_periods=100).mean().dropna() # 100개의 데이터가 하나의 묶음으로 (100일 이 다음날에 영향을 미친다.)
ts.head()
ts.tail() 

DatetimeIndex(['1999-10-01', '1999-10-02', '1999-10-03', '1999-10-04',
               '1999-10-05', '1999-10-06', '1999-10-07', '1999-10-08',
               '1999-10-09', '1999-10-10',
               ...
               '2002-09-25', '2002-09-26', '2002-09-27', '2002-09-28',
               '2002-09-29', '2002-09-30', '2002-10-01', '2002-10-02',
               '2002-10-03', '2002-10-04'],
              dtype='datetime64[ns]', length=1100, freq='D')


2002-09-30    0.889699
2002-10-01    0.894952
2002-10-02    0.859692
2002-10-03    0.837229
2002-10-04    0.818571
Freq: D, dtype: float64

In [42]:
transformed = (ts.groupby(lambda x:x.year).transform(lambda x:(x-x.mean())/x.std()))
transformed

2000-01-08    2.376784
2000-01-09    2.439961
2000-01-10    2.614934
2000-01-11    2.275308
2000-01-12    2.267149
                ...   
2002-09-30    1.877739
2002-10-01    1.899237
2002-10-02    1.754932
2002-10-03    1.663003
2002-10-04    1.586643
Freq: D, Length: 1001, dtype: float64

In [43]:
grouped = ts.groupby(lambda x : x.year)
grouped.mean()

2000    0.390128
2001    0.188152
2002    0.430885
dtype: float64

In [44]:
grouped_trans = transformed.groupby(lambda x: x.year)
grouped_trans.mean()

2000   -2.657732e-15
2001   -1.444811e-16
2002   -2.084173e-16
dtype: float64

In [45]:
sf = pd.Series([1,1,2,3,3,3])
sf.groupby(sf).filter(lambda x:x.sum()>3) # 합계를 낸게 2보다 큰

3    3
4    3
5    3
dtype: int64

In [53]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df 

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [57]:
df
df.groupby('key').aggregate(['min',np.median,max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [58]:
df.groupby('key').aggregate({'data1':'min','data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


In [59]:
df.groupby('key').transform(lambda x: x-x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


In [60]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x 

display('df',"df.groupby('key').apply(norm_by_data2)")

'df'

"df.groupby('key').apply(norm_by_data2)"

In [61]:
df = pd.DataFrame({'A' : ['일반', '일반', '특수', '예외'] * 6,
                   'B' : ['서울', '대구', '부산'] * 8,
                   'C' : ['건조', '건조', '건조', '우기', '우기', '우기'] * 4,
                   'D' : np.random.randn(24),
                   'E' : np.random.randn(24)})
print(df) 

     A   B   C         D         E
0   일반  서울  건조 -0.437026 -0.053294
1   일반  대구  건조 -0.630080  0.427418
2   특수  부산  건조 -1.169678  0.905786
3   예외  서울  우기  0.432308 -1.233786
4   일반  대구  우기  0.478399  0.540404
5   일반  부산  우기  0.045268 -0.788527
6   특수  서울  건조 -1.011696 -0.458976
7   예외  대구  건조  1.682640  1.575748
8   일반  부산  건조  0.613948  0.190385
9   일반  서울  우기  0.043308  2.256362
10  특수  대구  우기 -1.749464  0.288755
11  예외  부산  우기  1.363752  1.180851
12  일반  서울  건조 -0.860291 -1.246736
13  일반  대구  건조 -0.553085  0.000561
14  특수  부산  건조 -0.462079 -0.823885
15  예외  서울  우기 -1.157074  0.482545
16  일반  대구  우기 -1.832607  1.033994
17  일반  부산  우기 -0.976009  1.633767
18  특수  서울  건조  1.115703 -1.715448
19  예외  대구  건조 -0.501111  1.987521
20  일반  부산  건조  0.437238 -0.772051
21  일반  서울  우기 -1.444467  0.799228
22  특수  대구  우기  0.409517  0.369702
23  예외  부산  우기 -2.116385 -1.301871


In [65]:
df.A.unique() # 범주형 

array(['일반', '특수', '예외'], dtype=object)

In [67]:
 pd.crosstab(df.A,df.B,margins=True) # count , 교차분석표 예외 > 대구 ,예외> 부산 ...~ margins= True >합계

B,대구,부산,서울,All
A,,,,
예외,2,2,2,6
일반,4,4,4,12
특수,2,2,2,6
All,8,8,8,24


In [70]:
pd.crosstab([df.A,df.B],df.C, margins=True) # A,B 는 계층적 LEVEL1 LEVEL2 

C       건조  우기  All
A   B              
예외  대구   2   0    2
    부산   0   2    2
    서울   0   2    2
일반  대구   2   2    4
    부산   2   2    4
    서울   2   2    4
특수  대구   0   2    2
    부산   2   0    2
    서울   2   0    2
All     12  12   24

In [71]:
pd.crosstab(df.A, df.B).apply(lambda r : r/r.sum(), axis=1)

B,대구,부산,서울
A,,,
예외,0.333333,0.333333,0.333333
일반,0.333333,0.333333,0.333333
특수,0.333333,0.333333,0.333333


In [72]:
d = {
    'Name':['영구','일구','이구','삼구','사구','오구', '육구','칠구','팔구','박진','비옴','최훈'],
    'Exam':['중간고사','중간고사','중간고사','중간고사','중간고사','중간고사', '기말고사','기말고사','기말고사','기말고사','기말고사','기말고사'],
    
    'Subject':['수학','수학','수학','과학','과학','과학',  '수학','수학','수학','과학','과학','과학'],
    'Result':['Pass','Pass','Fail','Pass','Fail','Pass','Pass','Fail','Fail','Pass','Pass','Fail'],
    'Jumsu' : [ 100,80,70,60.,70,80,100,80,60,100,80,80]

} 

In [ ]:
- 1) 과목별 패스 여부를 확인하시오
- 2) 과목별로 분류하고 시험분별로 분류된 패스 여부를 확인 하고 합계 출력
- 3) 시험별로 패스 여부를 확인하시오
- 4) 시험별 이름별로 점수를 확인하시오 (범주형일때는 카운팅)
- 5) 시험별 과목별 점수를 확인하시오

In [126]:
df = pd.DataFrame(d)
df
#1
x1 = pd.crosstab(df.Subject,df.Result) 
x1

#2
x2 = pd.crosstab([df.Subject,df.Exam],df.Result, margins=True)
x2

#3
pd.crosstab(df.Exam,df.Result)

print(df)
#4
x4 = pd.crosstab([df.Name,df.Exam],df.Jumsu, margins=True)
x4

#5 
x5 = pd.crosstab(df.Exam, df.Subject, df.Jumsu, aggfunc=[np.average], margins=True)

   Name  Exam Subject Result  Jumsu
0    영구  중간고사      수학   Pass  100.0
1    일구  중간고사      수학   Pass   80.0
2    이구  중간고사      수학   Fail   70.0
3    삼구  중간고사      과학   Pass   60.0
4    사구  중간고사      과학   Fail   70.0
5    오구  중간고사      과학   Pass   80.0
6    육구  기말고사      수학   Pass  100.0
7    칠구  기말고사      수학   Fail   80.0
8    팔구  기말고사      수학   Fail   60.0
9    박진  기말고사      과학   Pass  100.0
10   비옴  기말고사      과학   Pass   80.0
11   최훈  기말고사      과학   Fail   80.0


In [102]:
# 피벗테이블 > 값이 나옴 크로스탭(카운팅)
df = pd.DataFrame({"foo": ['one', 'one-1', 'two', 'two'],
                    "bar": ['A', 'A', 'B', 'C'],
                    "baz": [1, 2, 3, 4]})
print(df)
df.pivot(index='foo', columns='bar', values='baz') 

     foo bar  baz
0    one   A    1
1  one-1   A    2
2    two   B    3
3    two   C    4


bar,A,B,C
foo,,,
one,1.0,NaN,NaN
one-1,2.0,NaN,NaN
two,NaN,3.0,4.0


In [104]:
data = {
    "도시": [  "서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"   ],
    "연도": [  "2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"    ],
    "인구": [  9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 2632035  ],
    "지역": [  "수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"    ]
} 


{'도시': ['서울', '서울', '서울', '부산', '부산', '부산', '인천', '인천'],
 '연도': ['2015', '2010', '2005', '2015', '2010', '2005', '2015', '2010'],
 '인구': [9904312,
  9631482,
  9762546,
  3448737,
  3393191,
  3512547,
  2890451,
  2632035],
 '지역': ['수도권', '수도권', '수도권', '경상권', '경상권', '경상권', '수도권', '수도권']}

In [108]:
# 도시 연도 별로 인구를 파악해보세요 
df = pd.DataFrame(data)
df
df.pivot(index='도시',columns='연도', values='인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


In [109]:
df = pd.read_excel('sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


In [112]:
df['Status'] = df['Status'].astype("category")


In [117]:
pd.pivot_table(df,index=['Name']) # > 숫자변수(연속형자료) 자동으로 

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000


In [125]:
pd.pivot_table(df, index=['Name'], columns=["Manager"])
pd.pivot_table(df, index=['Name'], columns=["Manager"],values=['Price'])
pd.pivot_table(df, index=['Manager','Rep'],values=["Price"], aggfunc = np.mean) # 집계를 할수있음 aggfunc = np.sum > 합계출력

Price
Manager       Rep                        
Debra Henley  Craig Booker   20000.000000
              Daniel Hilton  38333.333333
              John Smith     20000.000000
Fred Anderson Cedric Moss    27500.000000
              Wendy Yule     44250.000000